<img src="http://www.exalumnos.usm.cl/wp-content/uploads/2015/06/Isotipo-Negro.gif" title="Title text" width="20%" height="20%" />


<hr style="height:2px;border:none"/>
<h1 align='center'> INF-395/477 Redes Neuronales Artificiales I-2018 </h1>

<H3 align='center'> Tarea 2 - Aplicaciones Recientes de Redes Neuronales </H3>
<H3 align='center'> Pregunta 1 - RNN sobre texto</H3>
<hr style="height:2px;border:none"/>

**Librerías**

In [2]:
import numpy as np
import pandas as pd

**Dataset**

In [3]:
df_ner = pd.read_csv("./ner.csv", encoding ="cp1252", error_bad_lines=False)
df_ner.dropna(inplace=True)

b'Skipping line 281837: expected 25 fields, saw 34\n'


### a)

### b)

### c)

### d)

### e)

### f)

### g)

### h)

### i)

### j)

### k)

### l)